In [44]:
import pandas as pd
import numpy as np
import pyodbc
from slugify import slugify
import re

import pandas as pd
# import numpy as np 
import pyodbc
from slugify import slugify
from datetime import date

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

from pycaret.classification import setup, compare_models, tune_model, save_model, blend_models, create_model, calibrate_model, predict_model

from sklearn.metrics import f1_score,accuracy_score, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_selection import RFECV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import MinMaxScaler

import optuna
from optuna.pruners import MedianPruner
import logging
from joblib import Parallel, delayed

import numpy as np

from utils import send_email

np.random.seed(4407)  # Setting the numpy seed for global usage in this script
tf.random.set_seed(4407)  # for TensorFlow operations

In [2]:
df = pd.read_csv(r'C:\Users\bobby\Downloads\all_rotowire_stats.csv')

In [3]:
# Initialize an empty list to hold the names of columns to drop
columns_to_drop = []

# Initialize a set to keep track of seen (cleaned) column names
seen_columns = set()

# Iterate over the columns
for col in df.columns:
    # Remove trailing numbers and dots from column names to find the "base" name
    cleaned_col = col.rstrip('1234567890. ')
    
    # If the cleaned column name is already seen, mark the original column for removal
    if cleaned_col in seen_columns:
        columns_to_drop.append(col)
    else:
        # Otherwise, add the cleaned name to the set of seen column names
        seen_columns.add(cleaned_col)

# Drop the marked columns from the DataFrame
df = df.drop(columns=columns_to_drop)

In [4]:
server = 'localhost\SQLEXPRESS'
database = 'nba_game_data'

In [5]:
df

,Player,Position,Team,Opponent,Date,Time,Market Name,Line,Site Less,Site More,...,FanDuel Line,FanDuel Less,FanDuel More,FanDuel Less %,FanDuel More %,PointsBet Line,PointsBet Less,PointsBet More,PointsBet Less %,PointsBet More %
0,Kentavious Caldwell-Pope,G,DEN,@SAS,3/15/2024,7:30 PM,PTS+REB+AST,12.5,-122,-137,...,-,-,-,-,-,13.5,-130,100,53,47
1,Tre Jones,G,SAS,DEN,3/15/2024,7:30 PM,PTS+REB+AST,17.5,-121,-139,...,17.5,100,-128,47,53,17.5,-110,-120,49,51
2,Kentavious Caldwell-Pope,G,DEN,@SAS,3/15/2024,7:30 PM,Points,8.5,-136,-124,...,9.5,-132,104,54,46,8.5,-110,-120,49,51
3,Dejounte Murray,G,ATL,@UTA,3/15/2024,8:30 PM,PTS+REB+AST,41.5,-139,-121,...,40.5,-111,-115,50,50,40.5,-110,-120,49,51
4,Herbert Jones,F,NOP,LAC,3/15/2024,7:00 PM,PTS+REB+AST,14.5,-117,-143,...,-,-,-,-,-,15.5,-130,100,53,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,Nick Richards,C,CHA,@PHI,3/16/2024,6:00 PM,Assists,0.5,-137,-122,...,-,-,-,-,-,0.5,-125,-105,52,48
964,Jaren Jackson,C,MEM,OKC,3/16/2024,7:00 PM,PTS+REB+AST,32.5,-125,-136,...,32.5,-108,-118,49,51,32.5,-110,-120,49,51
965,Dillon Brooks,F,HOU,CLE,3/16/2024,4:00 PM,Rebounds,3.5,-130,-129,...,3.5,-118,-108,51,49,3.5,-115,-115,50,50
966,Fred VanVleet,G,HOU,CLE,3/16/2024,4:00 PM,Steals,1.5,-152,-112,...,1.5,-122,-104,52,48,-,-,-,-,-


In [6]:
df['line_Type'] = ''
for i, row in df.iterrows():
    if row['Market Name'] == 'Rebounds':
        df.loc[i,'line_Type'] ='REB'
    elif row['Market Name'] == '3PT Made':
        df.loc[i,'line_Type'] ='FG3M'
    elif row['Market Name'] == 'Assists':
        df.loc[i,'line_Type'] ='AST'
    elif row['Market Name'] == 'Blocks':
        df.loc[i,'line_Type'] ='BLK'
    elif row['Market Name'] == 'PTS+REB+AST':
        df.loc[i,'line_Type'] ='PTS+REB+AST'
    elif row['Market Name'] == 'Turnovers':
        df.loc[i,'line_Type'] ='TOV'
    elif row['Market Name'] == 'Points':
        df.loc[i,'line_Type'] ='PTS'
    elif row['Market Name'] == 'Steals':
        df.loc[i,'line_Type'] ='STL'
        
df['Opponent'] = df['Opponent'].apply(lambda x: x.replace('@','').strip())

In [7]:
df['Date'] = pd.to_datetime(df['Date'])
# Slugify player names for both DataFrames
df['Player_Slug'] = df['Player'].apply(slugify)


In [8]:
import pyodbc 
server = 'localhost\SQLEXPRESS'
database = 'nba_game_data'

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = """
SELECT 
      [PLAYER_NAME]
      ,PLAYER_ID
      ,[GAME_DATE]
      ,[REB]
      ,[AST]
      ,[TOV]
      ,[STL]
      ,[BLK]
      ,[PTS]
      ,FG3M

  FROM [nba_game_data].[dbo].[PlayerGameLogs]
  where yearSeason=2024
  """
df_actuals = pd.read_sql(sql,cnxn)



C:\Users\bobby\AppData\Local\Temp\ipykernel_15888\955121673.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_actuals = pd.read_sql(sql,cnxn)


In [9]:
df['Player_Slug'] = df['Player'].apply(slugify)
df_actuals['GAME_DATE'] = pd.to_datetime(df_actuals['GAME_DATE'])
df['Date'] = pd.to_datetime(df['Date'])
df_actuals['Player_Slug'] = df_actuals['PLAYER_NAME'].apply(slugify)
df_combined = pd.merge(df, df_actuals, how='left', left_on = ['Player_Slug','Date'],right_on=['Player_Slug','GAME_DATE'])

In [10]:
df_combined['actual'] = None
for i, row in df_combined.iterrows():
    if row['Market Name'] == 'Rebounds':
        df_combined.loc[i,'actual'] = df_combined.loc[i,'REB']
    elif row['Market Name'] == '3PT Made':
        df_combined.loc[i,'actual'] =df_combined.loc[i,'FG3M']
    elif row['Market Name'] == 'Assists':
        df_combined.loc[i,'actual'] =df_combined.loc[i,'AST']
    elif row['Market Name'] == 'Blocks':
        df_combined.loc[i,'actual'] =df_combined.loc[i,'BLK']
    elif row['Market Name'] == 'PTS+REB+AST':
        df_combined.loc[i,'actual'] =df_combined.loc[i,'PTS']+df_combined.loc[i,'REB']+df_combined.loc[i,'AST']
    elif row['Market Name'] == 'Turnovers':
        df_combined.loc[i,'actual'] =df_combined.loc[i,'TOV']
    elif row['Market Name'] == 'Points':
        df_combined.loc[i,'actual'] =df_combined.loc[i,'PTS']
    elif row['Market Name'] == 'Steals':
        df_combined.loc[i,'actual'] =df_combined.loc[i,'STL']

In [11]:
df_combined['over'] = (df_combined['actual']>df_combined['Line']).astype(int)

In [12]:
df_combined = df_combined.dropna(subset='actual')

In [13]:
# Function to clean column names
def clean_column_name(column_name):
    # Remove anything that is not a word character, a space, or a digit
    cleaned_name = re.sub(r"[^\w\s]", '', column_name)
    # Replace multiple spaces with a single space
    cleaned_name = re.sub(r"\s+", ' ', cleaned_name)
    return cleaned_name

# Clean all column names
df_combined.columns = [clean_column_name(col) for col in df_combined.columns]


In [14]:
df_combined['Lean'] = df_combined['Lean'].apply(lambda x: 1 if x =='More' else 0)

In [15]:
df_combined['At Home'] = df_combined['At Home'].apply(lambda x: 1 if x =='Yes' else 0)

In [16]:
# Columns you've identified as having meaningfully missing values
columns_with_missing_values = df_combined.columns#['Site Less', 'Site More', 'PrizePicks Line', 'Underdog Line', 'DraftKings Pick6 Line', 'BetMGM Line', 'BetRivers Line', 'DraftKings Line', 'FanDuel Line', 'PointsBet Line']

# Fill missing values with -1 for the specified columns
for column in columns_with_missing_values:
    df_combined[column] = df_combined[column].fillna(-1).replace('-',-1)


In [17]:
df_combined['Weighted Recent and Season Hit Rate'] = (
    0.6 * df_combined['Hit Rate Last 20 Outcomes'].astype(float).apply(lambda x: str(x).count('1')) / 20 +
    0.4 * df_combined['Hit Rate Previous Season'].astype(float)
)

In [18]:
columns = ['Hit Rate Last 10 Outcomes', 'Hit Rate Last 20 Outcomes', 'Hit Rate Last 30 Outcomes', 'Hit Rate Last 5 Outcomes']

# Loop through each column and calculate the sum of '1's for each row
for column in columns:
    # Assuming each cell is stored as a string of binary digits
    # If not, you might need to convert them to strings first
    df_combined[column] = df_combined[column].apply(lambda x: str(x).count('1'))

# Now, each of the specified columns has an accompanying column with the suffix ' Sum'
# that contains the sum of '1's, representing the number of games won.


In [19]:
df_combined = df_combined[['Market Name','Opponent','Line', 'Site Less', 'Site More', 'Prediction', 'Lean',
       'DFS Pickem Sites Factor', 'Sportsbooks Factor',
       'RotoWire Projection Factor', 'Hit Rate Factor', 'RotoWire Projection',
       'Weighted Hit Rate', 'At Home',
       'Average Line Variance', 'Implied Less ', 'Implied More ',
       'Hit Rate Last 10', 'Hit Rate Last 10 Outcomes',
       'Hit Rate Last 20 Outcomes', 'Hit Rate Last 30 Outcomes',
       'Hit Rate Last 5 Outcomes', 'Hit Rate Previous Season',
       'Hit Rate Season', 'Hit Rate Vs Opponent',
       'RotoWire Projection Difference', 'RotoWire Projection Difference ',
       'PrizePicks', 'PrizePicks Line', 'Underdog', 'Underdog Line',
       'DraftKings Pick6', 'DraftKings Pick6 Line', 'BetMGM Line',
       'BetMGM Less', 'BetMGM More',
       'BetRivers Line', 'BetRivers Less', 'BetRivers More',
       'DraftKings Line', 'DraftKings Less',
       'BetMGM More ', 'BetRivers More ', 'DraftKings More ', 'FanDuel More ',
       'DraftKings More',
       'FanDuel Line', 'FanDuel Less', 'FanDuel More',
       'PointsBet Line', 'PointsBet Less', 'PointsBet More','BetMGM Less ', 'BetRivers Less ', 'DraftKings Less ', 'FanDuel Less ',
       'over']]

In [20]:
df_combined['Performance Variability Vs Season'] = abs(df_combined['Hit Rate Vs Opponent'].astype(int) - df_combined['Hit Rate Season'].astype(int))
df_combined['Home Advantage Factor'] = df_combined['At Home'] * df_combined['Hit Rate Season']
df_combined['Projection Difference Momentum'] = df_combined['RotoWire Projection Difference'] - df_combined['RotoWire Projection Difference ']
df_combined['RotoWire Projection Difference']
df_combined['Predictive Confidence'] = (df_combined['Prediction'].astype(float) + df_combined['Sportsbooks Factor'].astype(float) + df_combined['DFS Pickem Sites Factor'].astype(float)) / 3
df_combined['Recent Performance Trend'] = df_combined['Hit Rate Last 5 Outcomes'].apply(lambda x: str(x).count('1')) / 5 - df_combined['Hit Rate Last 30 Outcomes'].apply(lambda x: str(x).count('1')) / 30
df_combined['Site Lean Disagreement'] = (df_combined['Site More'] - df_combined['Site Less']).abs()
df_combined['Implied Probability Discrepancy'] = (df_combined['Implied More '] - df_combined['Implied Less ']).abs()


In [21]:
# Replace -1 with NaN for calculation purposes in the specified columns
betting_columns_less = ['BetMGM Less ', 'BetRivers Less ', 'DraftKings Less ', 'FanDuel Less ']
betting_columns_more = ['BetMGM More ', 'BetRivers More ', 'DraftKings More ', 'FanDuel More ']
line_columns = ['BetMGM Line', 'BetRivers Line', 'DraftKings Line', 'FanDuel Line', 'PointsBet Line']

# Apply replacement
df_temp_less = df_combined[betting_columns_less].replace(-1, np.nan).astype(float)
df_temp_more = df_combined[betting_columns_more].replace(-1, np.nan).astype(float)
df_temp_lines = df_combined[line_columns].replace(-1, np.nan).astype(float)

# Calculate the mean for "less" and "more" consensus, ignoring NaN values
df_combined['Betting Consensus Less'] = df_temp_less.mean(axis=1)
df_combined['Betting Consensus More'] = df_temp_more.mean(axis=1)

# Calculate the variance in betting lines across sites, ignoring NaN values
df_combined['Betting Line Variance'] = df_temp_lines.var(axis=1)

In [23]:
df_combined = df_combined.fillna(-1)
# Replace infinite values with -1
df_combined = df_combined.replace([np.inf, -np.inf], -1)


In [24]:
df_combined = df_combined.drop(columns=['Opponent'])

In [25]:
df_just_market = df_combined.loc[df_combined['Market Name']=='Points']

In [26]:
df_just_market= df_just_market.drop(columns=['Market Name'])#['over']
df_just_market

,Line,Site Less,Site More,Prediction,Lean,DFS Pickem Sites Factor,Sportsbooks Factor,RotoWire Projection Factor,Hit Rate Factor,RotoWire Projection,...,Performance Variability Vs Season,Home Advantage Factor,Projection Difference Momentum,Predictive Confidence,Recent Performance Trend,Site Lean Disagreement,Implied Probability Discrepancy,Betting Consensus Less,Betting Consensus More,Betting Line Variance
2,8.5,-136,-124,22.0,1,0,7.5,66.9,62.5,10.51,...,37,0,-21.60,9.833333,0.0,12,2,51.333333,48.666667,0.25
5,20.5,-137,-124,-20.5,0,0,-6.5,-69.3,-52.5,18.42,...,22,0,8.06,-9.000000,0.0,13,2,50.000000,50.000000,0.70
11,24.5,-141,-120,-16.3,0,0,-2.5,-100.0,-2.5,20.68,...,29,0,11.78,-6.266667,0.0,21,4,52.000000,48.000000,0.00
14,22.5,-139,-122,-14.6,0,0,-0.2,-96.8,0.0,19.60,...,1,49,10.00,-4.933333,0.0,17,2,49.750000,50.250000,0.00
15,11.5,-150,-113,-14.5,0,-16.7,-30.9,-0.4,15.0,11.49,...,22,47,0.11,-20.700000,0.0,37,6,48.333333,51.666667,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,18.5,-141,-120,1.1,1,0,-10,-36.8,67.5,17.39,...,30,0,4.87,-2.966667,0.0,21,4,52.000000,48.000000,0.25
943,13.5,-132,-127,1.1,1,0,2.2,29.5,-27.5,14.39,...,0,50,-5.67,1.100000,0.2,5,0,48.666667,51.333333,0.00
950,19.5,-121,-139,-0.7,0,0,2.4,-8.2,-2.5,19.25,...,9,51,1.01,0.566667,0.0,18,4,48.666667,51.333333,0.00
952,22.5,-121,-139,0.7,1,0,7.7,1.8,-15.0,22.55,...,7,0,-0.19,2.800000,0.0,18,4,49.500000,50.500000,0.20


In [27]:
numeric_columns = df_just_market.select_dtypes(include=['int64', 'float64']).columns
df_just_market[numeric_columns] = df_just_market[numeric_columns].apply(pd.to_numeric, errors='coerce', downcast='float')


In [28]:
categorical_columns = df_just_market.select_dtypes(include=['object']).columns
df_just_market[categorical_columns] = df_just_market[categorical_columns].astype('category')


In [29]:
# Convert all columns in the DataFrame to float
df_just_market = df_just_market.apply(lambda x: x.replace(-1, np.nan).astype(float))
df_just_market.fillna(-1, inplace=True)
# If you want to convert -1 directly to float without replacing with NaN, you can skip the replacement part:
# df_just_market = df_just_market.astype(float)


In [42]:
X = df_just_market.drop(columns=['over'])
y=df_just_market['over']

In [47]:
 encoded_data, encoder, scaler = autoencode_features(X)

In [45]:
def autoencode_features(data, num_features=10):
    # Preprocess data: Normalize features
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data)

    # Dimensionality specifics
    input_dim = data_scaled.shape[1]
    encoding_dim = num_features

    # Building the autoencoder
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(encoding_dim, activation='relu')(input_layer)
    decoded = Dense(input_dim, activation='sigmoid')(encoded)
    autoencoder = Model(input_layer, decoded)

    # Encoder model
    encoder = Model(input_layer, encoded)

    # Compile autoencoder
    autoencoder.compile(optimizer=Adam(), loss='mean_squared_error')

    # Configure early stopping
    early_stopping = EarlyStopping(monitor='loss', min_delta=0.0001, patience=10, verbose=1, mode='auto')

    # Train the autoencoder
    autoencoder.fit(data_scaled, data_scaled, epochs=500, batch_size=256, shuffle=True, verbose=0, callbacks=[early_stopping], use_multiprocessing=True)

    # Encode the data
    encoded_data = encoder.predict(data_scaled, verbose=False, use_multiprocessing=True)

    return encoded_data, encoder, scaler

In [31]:
import numpy as np

# Calculate correlation matrix
corr_matrix = X.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features 
X_filtered = X.drop(to_drop, axis=1)


In [34]:
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier

# Initialize the model
model = RandomForestClassifier(random_state=42)

# Initialize RFECV
rfecv = RFECV(estimator=model, step=1, cv=5, scoring='accuracy', min_features_to_select=1,n_jobs =-1)

rfecv.fit(X_filtered, y)

# Transform the data
X_rfecv = rfecv.transform(X_filtered)

# Find the optimal number of features
print(f"Optimal number of features: {rfecv.n_features_}")


Optimal number of features: 5


In [35]:
model.fit(X_filtered, y)
feature_importances = pd.Series(model.feature_importances_, index=X_filtered.columns)

# Keep top n most important features, n determined from RFECV
n = rfecv.n_features_
top_n_features = feature_importances.nlargest(n).index

X_embedded = X_filtered[top_n_features]


In [36]:
# Assuming X_rfecv and X_embedded are your DataFrames after RFECV and Embedded method respectively
# Ensure that the features are in the form of DataFrame for easier handling
X_combined = pd.DataFrame(X_rfecv, columns=top_n_features)


In [37]:
combined = X_combined
combined['over'] = y.values

In [55]:
x_test = pd.concat([X_combined.drop(columns=['over']),pd.DataFrame(encoded_data)],axis=1)

In [39]:
from utils import PyroBayesianLogisticRegression, LeaveOneOutCrossValidationBobby,KFoldCrossValidationBobby

In [61]:
blr = PyroBayesianLogisticRegression(n_steps = 1000,lr=0.001, batch_size=100)
loocv = KFoldCrossValidationBobby(blr,n_jobs=-1,n_splits=15)
score = loocv.cross_validate(x_test,y)
score

0.295993450993451

In [65]:
import numpy as np
from joblib import Parallel, delayed
# from your_module import PyroBayesianLogisticRegression, KFoldCrossValidationBobby  # Ensure correct import

# Assuming X, y are defined somewhere above, and represent your dataset
X, y = np.array([...]), np.array([...])  # Replace with your actual data

# Define the grid of hyperparameters to search over
n_steps_options = [500, 1000, 1500]
lr_options = [0.001, 0.005, 0.01]
batch_size_options = [10, 50, len(X)]  # Adjust based on your dataset size

def evaluate_model(n_steps, lr, batch_size, X, y):
    try:
        """
        Function to train and evaluate the model given a set of hyperparameters.
        Returns a dictionary containing the hyperparameters and the corresponding score.
        """
        blr = PyroBayesianLogisticRegression(n_steps=n_steps, lr=lr, batch_size=batch_size)
        loocv = KFoldCrossValidationBobby(blr, n_jobs=2, n_splits=4)  # Adjust as needed
        score = loocv.cross_validate(x_test, y)
        return {'n_steps': n_steps, 'lr': lr, 'batch_size': batch_size, 'score': score}
    
    except:
        return {'n_steps': n_steps, 'lr': lr, 'batch_size': batch_size, 'score': 0}
    
# Use Joblib's Parallel and delayed to parallelize the evaluation across the grid
results = Parallel(n_jobs=-1, verbose=9)(
    delayed(evaluate_model)(n_steps, lr, batch_size, X, y)
    for n_steps in n_steps_options
    for lr in lr_options
    for batch_size in batch_size_options
)

# Find the best hyperparameter set based on the score
best_result = max(results, key=lambda x: x['score'])

print("Best Score:", best_result['score'])
print("Best Parameters:", best_result)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  27 | elapsed:   27.0s remaining:  2.6min
[Parallel(n_jobs=-1)]: Done   8 out of  27 | elapsed:   27.0s remaining:  1.1min
[Parallel(n_jobs=-1)]: Done  12 out of  27 | elapsed:   36.6s remaining:   45.7s
[Parallel(n_jobs=-1)]: Done  16 out of  27 | elapsed:   37.2s remaining:   25.6s
[Parallel(n_jobs=-1)]: Done  20 out of  27 | elapsed:   38.2s remaining:   13.3s


Best Score: 0
Best Parameters: {'n_steps': 500, 'lr': 0.001, 'batch_size': 10, 'score': 0}


[Parallel(n_jobs=-1)]: Done  24 out of  27 | elapsed:   38.4s remaining:    4.7s
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:   38.5s finished


In [407]:
from pycaret.classification import *

clf1 = setup(combined, target='over', session_id=123)


,Description,Value
0,Session id,123
1,Target,over
2,Target type,Binary
3,Original data shape,"(243, 6)"
4,Transformed data shape,"(243, 6)"
5,Transformed train set shape,"(170, 6)"
6,Transformed test set shape,"(73, 6)"
7,Numeric features,5
8,Preprocess,True
9,Imputation type,simple


In [408]:
best_model = compare_models()


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,0.5588,0.5590,0.5458,0.5380,0.5358,0.1180,0.1201,0.0210
dummy,Dummy Classifier,0.5235,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0260
knn,K Neighbors Classifier,0.5059,0.5403,0.4097,0.4830,0.4313,0.0038,0.0058,0.8530
svm,SVM - Linear Kernel,0.5000,0.0000,0.7917,0.4900,0.5969,0.0293,0.0067,0.0180
gbc,Gradient Boosting Classifier,0.5000,0.5167,0.4944,0.4824,0.4807,-0.0008,-0.0013,0.0610
et,Extra Trees Classifier,0.4882,0.4778,0.4069,0.4817,0.4239,-0.0321,-0.0283,0.1190
catboost,CatBoost Classifier,0.4824,0.4597,0.4083,0.4838,0.4214,-0.0417,-0.0327,2.6590
nb,Naive Bayes,0.4765,0.4069,0.2361,0.4086,0.2886,-0.0666,-0.0791,0.0210
xgboost,Extreme Gradient Boosting,0.4765,0.4875,0.4194,0.4496,0.4275,-0.0541,-0.0544,0.2030
rf,Random Forest Classifier,0.4647,0.4812,0.3958,0.4379,0.4044,-0.0779,-0.0758,0.1530


In [203]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Assume X and y are your features and target variable
# For demonstration, using placeholder variables
X = df_combined[[
    'Line', 'Site Less', 'Site More', 'Prediction', 'Lean',
    'DFS Pickem Sites Factor', 'Sportsbooks Factor', 'RotoWire Projection Factor', 
    'Hit Rate Factor', 'RotoWire Projection', 'Weighted Hit Rate', 'At Home', 
    'Average Line Variance', 'Implied Less ', 'Implied More ', 'Hit Rate Last 10', 
    'Hit Rate Last 10 Outcomes', 'Hit Rate Last 20 Outcomes', 'Hit Rate Last 30 Outcomes', 
    'Hit Rate Last 5 Outcomes', 'Hit Rate Previous Season', 'Hit Rate Season', 
    'Hit Rate Vs Opponent', 'RotoWire Projection Difference', 'RotoWire Projection Difference ',
    # Add other relevant features
]]
y = df_combined['over']  # Your binary target variable

# Generate interaction terms
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_interacted = poly.fit_transform(X)

# Get feature names for the interaction terms
feature_names = poly.get_feature_names(input_features=X.columns)


C:\Users\bobby\OneDrive\T-drive\Jupyter\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [204]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_interacted, y, test_size=0.3, random_state=42)

# Initialize and fit the RandomForest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Get feature importances
importances = model.feature_importances_


In [205]:
# Combine feature names and importances
feature_importance_dict = dict(zip(feature_names, importances))

# Sort features by importance
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# Filter out and list top interactions (excluding main effects)
top_interactions = [feature for feature, importance in sorted_features if " " in feature]

# Print top interactions
print("Top interactions:")
for interaction in top_interactions[:10]:  # Adjust the number to list more or fewer top interactions
    print(interaction)


Top interactions:
Prediction Sportsbooks Factor
Hit Rate Last 20 Outcomes Hit Rate Previous Season
RotoWire Projection Difference RotoWire Projection Difference 
Hit Rate Last 30 Outcomes Hit Rate Previous Season
Hit Rate Previous Season Hit Rate Season
Site More Hit Rate Previous Season
Site Less Hit Rate Previous Season
Line Hit Rate Vs Opponent
Site Less Site More
RotoWire Projection Hit Rate Season


In [211]:
# Example: Weighted sum of recent and previous season hit rates
df_combined['Weighted Recent and Season Hit Rate'] = (
    0.6 * df_combined['Hit Rate Last 20 Outcomes'].astype(float).apply(lambda x: str(x).count('1')) / 20 +
    0.4 * df_combined['Hit Rate Previous Season'].astype(float)
)

# Example: Trend in RotoWire Projection Difference
# This is more complex and might require calculating the change in projection difference over time,
# which could involve temporal data not shown here.



0      28.8
1      32.8
2      25.6
3       5.6
4      20.4
       ... 
962    19.2
963    17.2
965    19.2
966    21.2
967    20.4
Length: 876, dtype: float64

In [215]:
# Consensus across betting sites (simplified example)
# df_combined['Betting Consensus Less'] = 
df_combined[['BetMGM Less', 'BetRivers Less', 'DraftKings Less', 'FanDuel Less']].mean(axis=1)
# df_combined['Betting Consensus More'] = df_combined[['BetMGM More %', 'BetRivers More %', 'DraftKings More %', 'FanDuel More %']].mean(axis=1)

# # Divergence in betting lines across sites (variance)
# df_combined['Betting Line Variance'] = 
df_combined[['BetMGM Line', 'BetRivers Line', 'DraftKings Line', 'FanDuel Line', 'PointsBet Line']].var(axis=1)


C:\Users\bobby\AppData\Local\Temp\ipykernel_20176\3241764314.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_combined[['BetMGM Less', 'BetRivers Less', 'DraftKings Less', 'FanDuel Less']].mean(axis=1)
C:\Users\bobby\AppData\Local\Temp\ipykernel_20176\3241764314.py:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_combined[['BetMGM Line', 'BetRivers Line', 'DraftKings Line', 'FanDuel Line', 'PointsBet Line']].var(axis=1)


0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
962   NaN
963   NaN
965   NaN
966   NaN
967   NaN
Length: 876, dtype: float64

In [209]:
df_just_market['Prediction'] 
df_just_market['Sportsbooks Factor'] 


2        7.5
5       -6.5
11      -2.5
14      -0.2
15     -30.9
       ...  
941      -10
943      2.2
950      2.4
952      7.7
960      1.3
Name: Sportsbooks Factor, Length: 243, dtype: object

In [201]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import PolynomialFeatures

# Selecting features and target variable
X = df_combined[['Line', 'Site Less', 'Site More', 'Prediction', 'Lean', 'DFS Pickem Sites Factor', 'Sportsbooks Factor', 'RotoWire Projection Factor', 'Hit Rate Factor', 'RotoWire Projection', 'Weighted Hit Rate', 'At Home', 'Average Line Variance', 'Implied Less ', 'Implied More ', 'Hit Rate Last 10', 'Hit Rate Last 10 Outcomes', 'Hit Rate Last 20 Outcomes', 'Hit Rate Last 30 Outcomes', 'Hit Rate Last 5 Outcomes', 'Hit Rate Previous Season', 'Hit Rate Season', 'Hit Rate Vs Opponent', 'RotoWire Projection Difference', 'RotoWire Projection Difference ', 'PrizePicks', 'PrizePicks Line', 'Underdog', 'Underdog Line', 'DraftKings Pick6', 'DraftKings Pick6 Line', 'BetMGM Line', 'BetMGM Less', 'BetMGM More', 'BetMGM Less ', 'BetMGM More ', 'BetRivers Line', 'BetRivers Less', 'BetRivers More', 'BetRivers Less ', 'BetRivers More ', 'DraftKings Line', 'DraftKings Less', 'DraftKings More', 'DraftKings Less ', 'DraftKings More ', 'FanDuel Line', 'FanDuel Less', 'FanDuel More', 'FanDuel Less ', 'FanDuel More ', 'PointsBet Line', 'PointsBet Less', 'PointsBet More', 'PointsBet Less ', 'PointsBet More ']]
y = df_combined['over']

# Generate interaction terms
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_interacted = poly.fit_transform(X)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_interacted, y, test_size=0.3, random_state=42)

### Modeling with Random Forest (or choose another classifier)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict probabilities
y_pred_prob = model.predict_proba(X_test)[:, 1]

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f'ROC AUC Score: {roc_auc}')


ROC AUC Score: 0.5101832399626518


In [202]:
X_interacted

array([[ 1.250e+01, -1.220e+02, -1.370e+02, ...,  5.300e+03,  4.700e+03,
         2.491e+03],
       [ 1.750e+01, -1.210e+02, -1.390e+02, ..., -5.880e+03, -6.120e+03,
         2.499e+03],
       [ 8.500e+00, -1.360e+02, -1.240e+02, ..., -5.880e+03, -6.120e+03,
         2.499e+03],
       ...,
       [ 3.500e+00, -1.300e+02, -1.290e+02, ..., -5.750e+03, -5.750e+03,
         2.500e+03],
       [ 1.500e+00, -1.520e+02, -1.120e+02, ...,  1.000e+00,  1.000e+00,
         1.000e+00],
       [ 2.500e+00, -1.430e+02, -1.180e+02, ..., -5.610e+03, -5.390e+03,
         2.499e+03]])